# Case Study #7 Balanced Tree

The following are my solutions to the Case Study 7 Balanced Tree questions in 
[Danny Ma's Serious SQL course](https://www.datawithdanny.com/ "Data With Danny")
<br/>
<br/>
There are 4 [data tables](https://github.com/Shailesh-python/Case-Study-7-Balanced-Tree/blob/main/Data%20Sets) available to us in `balanced_tree` schema which we can use to run our SQL queries with:

1. `Product Details`

2. `Product Sales`

3. `Product Hierarcy`

4. `Product Price`

In [1]:
import pandas as pd
import numpy as np
import pyodbc as py
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
conn = py.connect(
    "DRIVER={SQL Server};SERVER=SHAILESH-PC\SQLEXPRESS;DATABASE=DannyMa;"
)

df_sales = pd.read_sql_query('select * from balanced_tree.sales',conn)
df_pp = pd.read_sql_query('select * from balanced_tree.product_prices',conn)
df_ph = pd.read_sql_query('select * from balanced_tree.product_hierarchy',conn)
df_pd = pd.read_sql_query('select * from balanced_tree.product_details',conn)

conn.close()

In [3]:
df_sales.head()

,prod_id,qty,price,discount,member,txn_id,start_txn_time
0,c4a632,4,13,17,True,54f307,2021-02-13 01:59:43.296
1,5d267b,4,40,17,True,54f307,2021-02-13 01:59:43.296
2,b9a74d,4,17,17,True,54f307,2021-02-13 01:59:43.296
3,2feb6b,2,29,17,True,54f307,2021-02-13 01:59:43.296
4,c4a632,5,13,21,True,26cc98,2021-01-19 01:39:00.3456


In [4]:
df_pd.head()

,product_id,price,product_name,category_id,segment_id,style_id,category_name,segment_name,style_name
0,c4a632,13,Navy Oversized Jeans - Womens,1,3,7,Womens,Jeans,Navy Oversized
1,e83aa3,32,Black Straight Jeans - Womens,1,3,8,Womens,Jeans,Black Straight
2,e31d39,10,Cream Relaxed Jeans - Womens,1,3,9,Womens,Jeans,Cream Relaxed
3,d5e9a6,23,Khaki Suit Jacket - Womens,1,4,10,Womens,Jacket,Khaki Suit
4,72f5d4,19,Indigo Rain Jacket - Womens,1,4,11,Womens,Jacket,Indigo Rain


In [5]:
df_ph.head()

,id,parent_id,level_text,level_name
0,1,NaN,Womens,Category
1,2,NaN,Mens,Category
2,3,1.0,Jeans,Segment
3,4,1.0,Jacket,Segment
4,5,2.0,Shirt,Segment


In [6]:
df_pp.head()

,id,product_id,price
0,7,c4a632,13
1,8,e83aa3,32
2,9,e31d39,10
3,10,d5e9a6,23
4,11,72f5d4,19


## Part A. High Level Sales Analysis

#### 1. What was the total quantity sold for all products?

In [7]:
col = list(df_sales.columns)
col[0] = 'product_id'
df_sales.columns = col
df_sales.columns

Index(['product_id', 'qty', 'price', 'discount', 'member', 'txn_id',
       'start_txn_time'],
      dtype='object')

In [8]:
df_sales.head()

,product_id,qty,price,discount,member,txn_id,start_txn_time
0,c4a632,4,13,17,True,54f307,2021-02-13 01:59:43.296
1,5d267b,4,40,17,True,54f307,2021-02-13 01:59:43.296
2,b9a74d,4,17,17,True,54f307,2021-02-13 01:59:43.296
3,2feb6b,2,29,17,True,54f307,2021-02-13 01:59:43.296
4,c4a632,5,13,21,True,26cc98,2021-01-19 01:39:00.3456


In [9]:
df = df_sales.merge(df_pd,on = 'product_id', how='left')
df.groupby(['product_name'])['qty'].sum().to_frame()

,qty
product_name,
Black Straight Jeans - Womens,3786
Blue Polo Shirt - Mens,3819
Cream Relaxed Jeans - Womens,3707
Grey Fashion Jacket - Womens,3876
Indigo Rain Jacket - Womens,3757
Khaki Suit Jacket - Womens,3752
Navy Oversized Jeans - Womens,3856
Navy Solid Socks - Mens,3792
Pink Fluro Polkadot Socks - Mens,3770


#### 2. What is the total generated revenue for all products before discounts?

In [10]:
df_2 = df
df_2['revenue'] = df_2['qty'] * df_2['price_x']
df_2.groupby(['product_name'])['revenue'].sum().to_frame()

,revenue
product_name,
Black Straight Jeans - Womens,121152
Blue Polo Shirt - Mens,217683
Cream Relaxed Jeans - Womens,37070
Grey Fashion Jacket - Womens,209304
Indigo Rain Jacket - Womens,71383
Khaki Suit Jacket - Womens,86296
Navy Oversized Jeans - Womens,50128
Navy Solid Socks - Mens,136512
Pink Fluro Polkadot Socks - Mens,109330


#### 3. What was the total discount amount for all products?

In [11]:
df.groupby(['product_name'])['discount'].sum().to_frame()

,discount
product_name,
Black Straight Jeans - Womens,15257
Blue Polo Shirt - Mens,15553
Cream Relaxed Jeans - Womens,15065
Grey Fashion Jacket - Womens,15500
Indigo Rain Jacket - Womens,15283
Khaki Suit Jacket - Womens,14669
Navy Oversized Jeans - Womens,15418
Navy Solid Socks - Mens,15646
Pink Fluro Polkadot Socks - Mens,14946


## Part B. Transaction Analysis

#### 1. How many unique transactions were there?

In [12]:
df_sales['txn_id'].drop_duplicates().count()

2500

#### 2. What is the average unique products purchased in each transaction?

In [13]:
df_sales[['txn_id','product_id']].drop_duplicates().groupby(['txn_id']).count().head()

,product_id
txn_id,
000dd8,6
003c6d,7
003ea6,4
0053d3,5
00a68b,7


#### 3. What are the 25th, 50th and 75th percentile values for the revenue per transaction?

In [14]:
arr=np.array(df_2.groupby(['txn_id'])['revenue'].sum().to_frame()['revenue'].sort_values())

In [15]:
print('25th percetile : ',np.percentile(arr, 25))
print('50th percetile : ',np.percentile(arr, 50))
print('75th percetile : ',np.percentile(arr, 75))

25th percetile :  375.75
50th percetile :  509.5
75th percetile :  647.0


#### 4. What is the average discount value per transaction?

In [16]:
df_2.groupby(['txn_id'])['discount'].mean().to_frame().head()

,discount
txn_id,
000dd8,6.0
003c6d,13.0
003ea6,6.0
0053d3,20.0
00a68b,7.0


#### 5. What is the percentage split of all transactions for members vs non-members?

In [17]:
df_5 = df_sales
df_5['is_member']=df_5['member'].map({True : 'Yes', False : 'No'})
df_5 = df_5[['txn_id','is_member']].groupby(['is_member']).count()
total_transactions = df_5['txn_id'].sum()
df_5['precent'] = 100*(df_5['txn_id']/total_transactions)
df_5

,txn_id,precent
is_member,,
No,6034,39.973501
Yes,9061,60.026499


#### 6. What is the average revenue for member transactions and non-member transactions?

In [18]:
df_5 = df_sales
df_5['is_member']=df_5['member'].map({True : 'Yes', False : 'No'})
df_5['revenue']=df_5['qty'] * df_5['price']
df_5[['revenue','is_member']].groupby(['is_member'])['revenue'].mean()

is_member
No     84.930229
Yes    85.750359
Name: revenue, dtype: float64

## Part C. Product Analysis

#### 1.What are the top 3 products by total revenue before discount?

In [19]:
top3 = df.groupby(['product_id','product_name'])['revenue'].sum().to_frame()['revenue'].sort_values(ascending=False).to_frame()
top3.head(3)

,,revenue
product_id,product_name,
2a2353,Blue Polo Shirt - Mens,217683
9ec847,Grey Fashion Jacket - Womens,209304
5d267b,White Tee Shirt - Mens,152000


#### 2. What is the total quantity, revenue and discount for each segment?

In [20]:
df.groupby(['segment_name'])[['segment_name','qty','discount','revenue']].sum()

,qty,discount,revenue
segment_name,,,
Jacket,11385,45452,366983
Jeans,11349,45740,208350
Shirt,11265,46043,406143
Socks,11217,45465,307977


#### 3.What is the top selling product for each segment?

In [21]:
df.groupby(['segment_name','product_name'])['qty'].sum().to_frame()

qty
segment_name product_name                          
Jacket       Grey Fashion Jacket - Womens      3876
             Indigo Rain Jacket - Womens       3757
             Khaki Suit Jacket - Womens        3752
Jeans        Black Straight Jeans - Womens     3786
             Cream Relaxed Jeans - Womens      3707
             Navy Oversized Jeans - Womens     3856
Shirt        Blue Polo Shirt - Mens            3819
             Teal Button Up Shirt - Mens       3646
             White Tee Shirt - Mens            3800
Socks        Navy Solid Socks - Mens           3792
             Pink Fluro Polkadot Socks - Mens  3770
             White Striped Socks - Mens        3655

#### 4. What is the total quantity, revenue and discount for each category?

In [22]:
df.groupby(['category_name'])[['segment_name','qty','discount','revenue']].sum()

,qty,discount,revenue
category_name,,,
Mens,22482,91508,714120
Womens,22734,91192,575333


#### 5.What is the top selling product for each category?

In [23]:
topsellingproduct = df.groupby(['category_name','product_name'])['revenue'].sum().to_frame()
topsellingproduct['rn'] = topsellingproduct.groupby('category_name').revenue.rank(method='dense',ascending=False)
topsellingproduct[topsellingproduct['rn']==1]

,,revenue,rn
category_name,product_name,,
Mens,Blue Polo Shirt - Mens,217683,1.0
Womens,Grey Fashion Jacket - Womens,209304,1.0


#### 6. What is the percentage split of revenue by product for each segment?

In [24]:
dff = df.groupby(['segment_name','product_name'])['revenue'].sum().to_frame()
dff

revenue
segment_name product_name                             
Jacket       Grey Fashion Jacket - Womens       209304
             Indigo Rain Jacket - Womens         71383
             Khaki Suit Jacket - Womens          86296
Jeans        Black Straight Jeans - Womens      121152
             Cream Relaxed Jeans - Womens        37070
             Navy Oversized Jeans - Womens       50128
Shirt        Blue Polo Shirt - Mens             217683
             Teal Button Up Shirt - Mens         36460
             White Tee Shirt - Mens             152000
Socks        Navy Solid Socks - Mens            136512
             Pink Fluro Polkadot Socks - Mens   109330
             White Striped Socks - Mens          62135

In [25]:
dff.reset_index(drop=False, inplace=True)

In [26]:
df_perc = dff.merge(dff.groupby(['segment_name'])['revenue'].sum().to_frame(),on='segment_name',how='left')

In [27]:
df_perc['split_perc'] = df_perc['revenue_x']/df_perc['revenue_y']

In [28]:
df_perc

,segment_name,product_name,revenue_x,revenue_y,split_perc
0,Jacket,Grey Fashion Jacket - Womens,209304,366983,0.570337
1,Jacket,Indigo Rain Jacket - Womens,71383,366983,0.194513
2,Jacket,Khaki Suit Jacket - Womens,86296,366983,0.235150
3,Jeans,Black Straight Jeans - Womens,121152,208350,0.581483
4,Jeans,Cream Relaxed Jeans - Womens,37070,208350,0.177922
5,Jeans,Navy Oversized Jeans - Womens,50128,208350,0.240595
6,Shirt,Blue Polo Shirt - Mens,217683,406143,0.535976
7,Shirt,Teal Button Up Shirt - Mens,36460,406143,0.089771
8,Shirt,White Tee Shirt - Mens,152000,406143,0.374252
9,Socks,Navy Solid Socks - Mens,136512,307977,0.443254


#### 7.What is the percentage split of revenue by segment for each category?

In [36]:
dff = df.groupby(['segment_name','category_name'])['revenue'].sum().to_frame()
dff.reset_index(drop=False, inplace=True)
dff

,segment_name,category_name,revenue
0,Jacket,Womens,366983
1,Jeans,Womens,208350
2,Shirt,Mens,406143
3,Socks,Mens,307977


In [38]:
df_perc = dff.merge(dff.groupby(['category_name'])['revenue'].sum().to_frame(),on='category_name',how='left')

In [39]:
df_perc['split_perc'] = df_perc['revenue_x']/df_perc['revenue_y']

In [40]:
df_perc

,segment_name,category_name,revenue_x,revenue_y,split_perc
0,Jacket,Womens,366983,575333,0.637862
1,Jeans,Womens,208350,575333,0.362138
2,Shirt,Mens,406143,714120,0.568732
3,Socks,Mens,307977,714120,0.431268


#### 8. What is the percentage split of revenue by each category?

In [42]:
dff = df.groupby(['category_id','category_name'])['revenue'].sum().to_frame()
dff.reset_index(drop=False, inplace=True)
dff

,category_id,category_name,revenue
0,1,Womens,575333
1,2,Mens,714120


In [51]:
dff['split_perc'] = dff['revenue']/dff['revenue'].sum()

In [52]:
dff

,category_id,category_name,revenue,split_perc
0,1,Womens,575333,0.446184
1,2,Mens,714120,0.553816
